In [1]:
import json
import numpy as np
import torch
import scipy.sparse as sparse
from ogb.graphproppred import GraphPropPredDataset
from torch_geometric.loader import DataLoader

# Download and process data at './dataset/ogbg_molclintox/'
dataset = GraphPropPredDataset(name = 'ogbg-molclintox')


split_idx = dataset.get_idx_split()
train_idx, valid_idx, test_idx = split_idx["train"], split_idx["valid"], split_idx["test"]

/home/jimmyzxj/miniconda3/envs/gli_env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(dataset[0][0].keys())

dict_keys(['edge_index', 'edge_feat', 'node_feat', 'num_nodes'])


In [3]:
from tqdm import tqdm
node_list = []
labels = []
edges = []
edge_feats = []
node_feats = []
num_nodes = 0

In [4]:
for g, label in tqdm(dataset):
    node_list.append(np.arange(g["num_nodes"]) + num_nodes)  # All the nodes are considered in a single graph
    edges.append(np.stack(g["edge_index"]).T + num_nodes)
    edge_feats.append(g["edge_feat"])
    node_feats.append(g["node_feat"])
    labels.append(label)
    num_nodes += g["num_nodes"]

100%|██████████| 1477/1477 [00:00<00:00, 36223.76it/s]


In [5]:
_edges = np.concatenate(edges, axis=0)
_labels = np.stack(labels).squeeze()
_edge_feats = np.concatenate(edge_feats)
_node_feats = np.concatenate(node_feats)

In [6]:
_node_list = sparse.csr_matrix((len(dataset), num_nodes))
for i, indices in enumerate(node_list):
    _node_list[i, indices] = 1

/home/jimmyzxj/miniconda3/envs/gli_env/lib/python3.8/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [7]:
from gli.utils import save_data
data = {
    "node_feats": _node_feats,
    "edge": _edges,
    "graph_class": _labels,
    "edge_feats": _edge_feats,
    "node_list": _node_list,
}
save_data("ogbg-molclintox", **data)

Save all dense arrays to ogbg-molclintox.npz, including ['node_feats', 'edge', 'graph_class', 'edge_feats']
Save sparse matrix node_list to ogbg-molclintox_node_list.sparse.npz


In [9]:
task_data={
    "train" :train_idx,
    "val" :valid_idx,
    "test" :test_idx
}
save_data("ogbg-molclintox_task",**task_data)

Save all dense arrays to ogbg-molclintox_task.npz, including ['train', 'val', 'test']
